In [10]:
import pandas as pd
from io import StringIO

QUEUE_NAMES = {
    'VS-case-inbox-spa-ES-tier2': 'Vendor Tier2',
    'CS-case-inbox-spa-ES-tier2': 'Customer Tier2',
    'RS-case-inbox-spa-ES-tier2': 'Rider Tier2',
    'RS-chat-spa-ES-tier1': 'Rider Tier1',
    'VS-chat-spa-ES-tier1': 'Vendor Tier1',
    'CS-chat-spa-ES-live-order': 'Customer Live',
    'CS-chat-spa-ES-nonlive-order': 'Customer Non Live'
}

def clean_contacts_with_ccr(data: pd.DataFrame):

    # ─────────────────────
    # Limpieza básica
    # ─────────────────────
    data = data.iloc[:-1]
    data = data[data['queue_name'].isin(QUEUE_NAMES.keys())].copy()

    # ─────────────────────
    # Timestamp (UTC)
    # ─────────────────────
    data['creation_timestamp_utc'] = pd.to_datetime(
        data['creation_timestamp_local'],
        format='%B %d, %Y, %I:%M %p',
        utc=True
    )

    # ─────────────────────
    # Zonas horarias
    # ─────────────────────
    data['timestamp_pe'] = data['creation_timestamp_utc'].dt.tz_convert('America/Lima')
    data['timestamp_es'] = data['creation_timestamp_utc'].dt.tz_convert('Europe/Madrid')

    data['date_pe'] = data['timestamp_pe'].dt.date
    data['interval_pe'] = data['timestamp_pe'].dt.strftime('%H:00')

    data['date_es'] = data['timestamp_es'].dt.date
    data['interval_es'] = data['timestamp_es'].dt.strftime('%H:00')

    # ─────────────────────
    # Team
    # ─────────────────────
    data['team'] = data['queue_name'].map(QUEUE_NAMES)

    # ─────────────────────
    # Contact reason
    # ─────────────────────
    data['contact_reason'] = data['cr_l1']
    data.loc[data['cr_l2'].notna(), 'contact_reason'] += '/' + data['cr_l2']
    data.loc[data['cr_l3'].notna(), 'contact_reason'] += '/' + data['cr_l3']

    # ─────────────────────
    # Tabla 1: métricas por intervalo
    # ─────────────────────
    interval_metrics = (
        data.groupby(
            ['date_pe', 'interval_pe', 'date_es', 'interval_es', 'team'],
            as_index=False
        )
        .size()
        .rename(columns={'size': 'received'})
    )

    # ─────────────────────
    # Tabla 2: conteo por CR
    # ─────────────────────
    cr_table = (
        data.groupby(
            ['date_pe', 'interval_pe', 'date_es', 'interval_es', 'team', 'contact_reason'],
            as_index=False
        )
        .size()
        .rename(columns={'size': 'count'})
    )

    return interval_metrics, cr_table

with open('contacts_with_ccr.csv', encoding='utf-8') as f:
    clean_csv = f.read().rsplit('\n', 1)[0]

df = pd.read_csv(
    StringIO(clean_csv),
    engine='python'
)
data1, data2 = clean_contacts_with_ccr(df)
print(data1.head(20))
print(data2.head(20))


       date_pe interval_pe     date_es interval_es               team  \
0   2025-12-15       09:00  2025-12-15       15:00      Customer Live   
1   2025-12-15       09:00  2025-12-15       15:00  Customer Non Live   
2   2025-12-15       09:00  2025-12-15       15:00     Customer Tier2   
3   2025-12-15       09:00  2025-12-15       15:00        Rider Tier1   
4   2025-12-15       09:00  2025-12-15       15:00        Rider Tier2   
5   2025-12-15       09:00  2025-12-15       15:00       Vendor Tier1   
6   2025-12-15       09:00  2025-12-15       15:00       Vendor Tier2   
7   2025-12-15       10:00  2025-12-15       16:00      Customer Live   
8   2025-12-15       10:00  2025-12-15       16:00  Customer Non Live   
9   2025-12-15       10:00  2025-12-15       16:00     Customer Tier2   
10  2025-12-15       10:00  2025-12-15       16:00        Rider Tier1   
11  2025-12-15       10:00  2025-12-15       16:00        Rider Tier2   
12  2025-12-15       10:00  2025-12-15       16:00 